In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

from modules.judge import judge  # 导入触发判断函数
from modules.info import *
from modules.save2csv import *


In [2]:
entity_fin = pd.read_csv(path + 'outputs/2_计算好指标的主体_' + date + '.csv')
threshold = pd.read_excel(path + 'rules/模型规则_230526.xlsx', sheet_name='阈值配置表')

In [3]:
# 生成额外带有偏移量的指标
entity_fin['MinProfitAndProfitAfterDedu^1'] = entity_fin['MinProfitAndProfitAfterDedu'].shift(1)
entity_fin['MinProfitAndProfitAfterDedu^2'] = entity_fin['MinProfitAndProfitAfterDedu'].shift(2)
entity_fin['NetProfit^1'] = entity_fin['NetProfit'].shift(1)

# TODO: 数据待补充
# entity_fin['ConsecutiveGoodAuditReports^1'] = entity_fin['ConsecutiveGoodAuditReports'].shift(1)
# entity_fin['ConsecutiveGoodAuditReports^2'] = entity_fin['ConsecutiveGoodAuditReports'].shift(2)


In [4]:
# 具体规则触发

threshold.rename(columns={'指标编码':'variable',
                          '具体规则编码':'speci_rules',
                          '触发类型-方向':'how',
                          '行业':'industry_whitewash',
                          '高危触发':'threshold_danger',
                          '普通触发':'threshold_normal'}, inplace=True)
threshold.drop(columns={'时间参数', '下限', '上限'}, inplace=True)

entity_fin = pd.melt(entity_fin, id_vars=['entity_id', 'year', 'entity_name', 'stock_code', 'stock_name', 'uniform_credit', 'industry_whitewash'])
entity_fin = pd.merge(entity_fin, threshold, on='variable')

entity_fin = entity_fin[(entity_fin['industry_whitewash_x'] == entity_fin['industry_whitewash_y']) | (entity_fin['industry_whitewash_y'] == '-')]

# 符号处理

entity_fin['how'] = entity_fin['how'].str.replace('等于', '==')
entity_fin['how'] = entity_fin['how'].str.replace('小于==', '<=')
entity_fin['how'] = entity_fin['how'].str.replace('大于==', '>=')
entity_fin['how'] = entity_fin['how'].str.replace('不==', '!=')

entity_fin['how'] = entity_fin['how'].str.replace('小于', '<')
entity_fin['how'] = entity_fin['how'].str.replace('大于', '>')

entity_fin['threshold_danger'] = pd.to_numeric(entity_fin['threshold_danger'])
entity_fin['threshold_normal'] = pd.to_numeric(entity_fin['threshold_normal'])

entity_fin['value'] = entity_fin['value'].astype(str).str.replace('inf', 'np.inf')
entity_fin['value'] = entity_fin['value'].astype(str).str.replace('nan', 'np.nan')

# 具体规则判断

entity_fin['result_danger'] = entity_fin.apply(lambda row: eval(f"{row['value']} {row['how']} {row['threshold_danger']}"), axis=1)
entity_fin['result_normal'] = entity_fin.apply(lambda row: eval(f"{row['value']} {row['how']} {row['threshold_normal']}"), axis=1)

# 缺失值默认不触发
entity_fin['result_danger'] = np.where(entity_fin['value'] == 'np.nan', False, entity_fin['result_danger'])
entity_fin['result_normal'] = np.where(entity_fin['value'] == 'np.nan', False, entity_fin['result_normal'])


entity_fin.drop(columns=['industry_whitewash_x', 'industry_whitewash_y', 'how', 'threshold_danger', 'threshold_normal', 'value'], inplace=True)


In [5]:
# 读取【具体规则_规则_法规】映射
speci_rule_map = pd.read_excel(path + 'rules/模型规则_230407.xlsx', sheet_name='具体规则_规则_法规')
speci_rule_map.drop(columns={'逻辑关系.1', '规则简称'}, inplace=True)
speci_rule_map.rename(columns={'具体规则编码':'speci_rules',
                               '逻辑关系':'logi',
                               '规则编码':'rules',
                               '逻辑关系.2':'logi2',
                               '法规编码':'law'}, inplace=True)
rule = pd.merge(entity_fin, speci_rule_map, on='speci_rules')

In [6]:
# 规则判断
rule_result_danger = judge(rule, ['entity_id','speci_rules'], 'result_danger', 'logi', 'rule_result_danger', rule, ['entity_id', 'speci_rules'])
entity_with_rule   = judge(rule, ['entity_id','speci_rules'], 'result_normal', 'logi', 'rule_result_normal', rule_result_danger, ['entity_id', 'speci_rules'])

# 法规判断
law_result_danger = judge(entity_with_rule, ['entity_id','rules'], 'rule_result_danger', 'logi2', 'law_result_danger', entity_with_rule, ['entity_id', 'rules'])
entity_with_law   = judge(entity_with_rule, ['entity_id','rules'], 'rule_result_normal', 'logi2', 'law_result_normal', law_result_danger, ['entity_id', 'rules'])


In [7]:
entity_with_law.drop(columns=['variable', 'speci_rules', 'result_danger', 'result_normal', 'logi', 'rules', 'logi2', 'rule_result_danger', 'rule_result_normal'], inplace=True)
entity_with_law.drop_duplicates(inplace=True)
entity_with_law

,entity_id,year,entity_name,stock_code,stock_name,uniform_credit,law,law_result_danger,law_result_normal
0,19718,2021,万科企业股份有限公司,000002.SZ,万科A,91440300192181490G,BSE-IPOLaw-2-2,False,False
1,19718,2020,万科企业股份有限公司,000002.SZ,万科A,91440300192181490G,BSE-IPOLaw-2-2,False,False
2,19718,2019,万科企业股份有限公司,000002.SZ,万科A,91440300192181490G,BSE-IPOLaw-2-2,False,False
12,19719,2021,深圳国华网安科技股份有限公司,000004.SZ,国农科技,91440300192441969E,BSE-IPOLaw-2-2,False,False
13,19719,2020,深圳国华网安科技股份有限公司,000004.SZ,国农科技,91440300192441969E,BSE-IPOLaw-2-2,False,False
...,...,...,...,...,...,...,...,...,...
1562732,20379,2019,大连天宝绿色食品股份有限公司,041566001.IB,15天宝食品CP001,912102006049208959,CSRC-IPOLaw-2-3,True,True
1562736,20831,2019,长生生物科技股份有限公司,400077.NQ,R长生1,91320700138991134G,CSRC-IPOLaw-2-3,True,True
1562740,21103,2019,金亚科技股份有限公司,400087.NQ,R金亚1,91510100720307596L,CSRC-IPOLaw-2-3,True,True
1562744,21499,2019,暴风集团股份有限公司,300431.SZ,暴风退,91110000798532048H,CSRC-IPOLaw-2-3,True,True


In [8]:
save2csv(entity_with_rule, '3.1_规则触发结果')
save2csv(entity_with_law, '3_法规触发结果')